In [1]:
import numpy as np 
import math
import matplotlib.pyplot as plt 

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.RK45.html#scipy.integrate.RK45
from scipy.integrate import odeint, RK45

# Target: We want to mimic the activity of a 3 neuron network with a HH neuron

<img src="./3_neuron_example.png" alt="drawing" width="900" />

Tasks:

1. Use simple HH neurons to build this network
2. Use different HH neurons to build this network
3. Use different DDF neurons to build this network


### First, code up a solution with non-modular code

Steps

1. define differential equation to calculate update values at different states
2. specify time bound, number of steps, and initial condition
3. use Runge-Kutta method to integrate forward in time
4. create each component's plot and attractor plot

Variables

- Each Neuron has 4 variables (m, h, n, V); in a 3 neurons, 4 synapses network, we have a total of 12 variables.

Here is a table:

| Variable Name | Math Notation | Description                     |
| :------------ | :------------ | :------------------------------ |
| m_1           | $m_1(t)$      | Neuron 1: Na gating variable #1 |
| h_1           | $h_1(t)$      | Neuron 1: Na gating variable #2 |
| n_1           | $n_1(t)$      | Neuron 1: K gating variable     |
| m_2           | $m_2(t)$      | Neuron 2: Na gating variable #1 |
| h_2           | $h_2(t)$      | Neuron 2: Na gating variable #2 |
| n_2           | $n_2(t)$      | Neuron 2: K gating variable     |
| m_3           | $m_3(t)$      | Neuron 3: Na gating variable #1 |
| h_3           | $h_3(t)$      | Neuron 3: Na gating variable #2 |
| n_3           | $n_3(t)$      | Neuron 3: K gating variable     |
| V_1           | $V_1(t)$      | Neuron 1: Voltage               |
| V_2           | $V_2(t)$      | Neuron 2: Voltage               |
| V_3           | $V_3(t)$      | Neuron 3: Voltage               |

Constants

- There are 3 gating constants for each neuron, and 1 gating constant for each synapse.
- In addition, each neuron has 4 field constants, and 2 voltage constants.

Here is another table:

<!-- FOR NEURON -->

#### NEURON 1

| Constant Name | Math Notation  | Description                    |
| :------------ | :------------- | :----------------------------- |
| g_Na_1        | $g_{Na_1}$     | Neuron 1: Na gating constant   |
| g_K_1         | $g_{K_1}$      | Neuron 1: K gating constant    |
| g_L_1         | $g_{L_1}$      | Neuron 1: Leak gating constant |
| E_Na_1        | $E_{Na_1}$     |                                |
| E_K_1         | $E_{K_1}$      |                                |
| E_L_1         | $E_{L_1}$      |                                |
| V0_m_1        | $V0_{m_1}$     |                                |
| V0_h_1        | $V0_{h_1}$     |                                |
| V0_n_1        | $V0_{n_1}$     |                                |
| dV0_m_1       | $dV0_{m_1}$    |                                |
| dV0_h_1       | $dV0_{h_1}$    |                                |
| dV0_n_1       | $dV0_{n_1}$    |                                |
| tau_m_1_0     | $\tau_{m_1-0}$ |                                |
| tau_m_1_1     | $\tau_{m_1-1}$ |                                |
| tau_h_1_0     | $\tau_{h_1-0}$ |                                |
| tau_h_1_1     | $\tau_{h_1-1}$ |                                |
| tau_n_1_0     | $\tau_{n_1-0}$ |                                |
| tau_n_1_1     | $\tau_{n_1-1}$ |                                |

#### NEURON 2

| Constant Name | Math Notation  | Description                    |
| :------------ | :------------- | :----------------------------- |
| g_Na_2        | $g_{Na_2}$     | Neuron 2: Na gating constant   |
| g_K_2         | $g_{K_2}$      | Neuron 2: K gating constant    |
| g_L_2         | $g_{L_2}$      | Neuron 2: Leak gating constant |
| E_Na_2        | $E_{Na_2}$     |                                |
| E_K_2         | $E_{K_2}$      |                                |
| E_L_2         | $E_{L_2}$      |                                |
| V0_m_2        | $V0_{m_2}$     |                                |
| V0_h_2        | $V0_{h_2}$     |                                |
| V0_n_2        | $V0_{n_2}$     |                                |
| dV0_m_2       | $dV0_{m_2}$    |                                |
| dV0_h_2       | $dV0_{h_2}$    |                                |
| dV0_n_2       | $dV0_{n_2}$    |                                |
| tau_m_2_0     | $\tau_{m_2-0}$ |                                |
| tau_m_2_1     | $\tau_{m_2-1}$ |                                |
| tau_h_2_0     | $\tau_{h_2-0}$ |                                |
| tau_h_2_1     | $\tau_{h_2-1}$ |                                |
| tau_n_2_0     | $\tau_{n_2-0}$ |                                |
| tau_n_2_1     | $\tau_{n_2-1}$ |                                |

#### NEURON 3

| Constant Name | Math Notation  | Description                    |
| :------------ | :------------- | :----------------------------- |
| g_Na_3        | $g_{Na_3}$     | Neuron 3: Na gating constant   |
| g_K_3         | $g_{K_3}$      | Neuron 3: K gating constant    |
| g_L_3         | $g_{L_3}$      | Neuron 3: Leak gating constant |
| E_Na_3        | $E_{Na_3}$     |                                |
| E_K_3         | $E_{K_3}$      |                                |
| E_L_3         | $E_{L_3}$      |                                |
| V0_m_3        | $V0_{m_3}$     |                                |
| V0_h_3        | $V0_{h_3}$     |                                |
| V0_n_3        | $V0_{n_3}$     |                                |
| dV0_m_3       | $dV0_{m_3}$    |                                |
| dV0_h_3       | $dV0_{h_3}$    |                                |
| dV0_n_3       | $dV0_{n_3}$    |                                |
| tau_m_3_0     | $\tau_{m_3-0}$ |                                |
| tau_m_3_1     | $\tau_{m_3-1}$ |                                |
| tau_h_3_0     | $\tau_{h_3-0}$ |                                |
| tau_h_3_1     | $\tau_{h_3-1}$ |                                |
| tau_n_3_0     | $\tau_{n_3-0}$ |                                |
| tau_n_3_1     | $\tau_{n_3-1}$ |                                |

#### FOR SYNAPSE

| Constant Name | Math Notation | Description                       |
| :------------ | :------------ | :-------------------------------- |
| g_12          | $g_12$        | Synapse 12: gating constant       |
| g_31          | $g_31$        | Synapse 31: gating constant       |
| g_23          | $g_23$        | Synapse 23: gating constant       |
| g_32          | $g_32$        | Synapse 32: gating constant       |
| V0_12         | $V0_12$       | Synapse 12: synapse voltage       |
| V0_31         | $V0_31$       | Synapse 31: synapse voltage       |
| V0_23         | $V0_23$       | Synapse 23: synapse voltage       |
| V0_32         | $V0_32$       | Synapse 32: synapse voltage       |
| dV0_12        | $dV0_12$      | Synapse 12: synapse voltage delta |
| dV0_31        | $dV0_31$      | Synapse 31: synapse voltage delta |
| dV0_23        | $dV0_23$      | Synapse 23: synapse voltage delta |
| dV0_32        | $dV0_32$      | Synapse 32: synapse voltage delta |

Shared Constants

| Constant Name | Math Notation | Description                                                            |
| :------------ | :------------ | :--------------------------------------------------------------------- |
| tau1          | $\tau_{1}$    | Neuron 1: excitatory postsynaptic tau constant at the denominator of S |
| tau2          | $\tau_{2}$    | Neuron 2: inhibitory postsynaptic tau constant at the denominator of S |
| E_rev_e       | $E_{rev_1}$   |                                                                        |
| E_rev_i       | $E_{rev_2}$   |                                                                        |
| alpha         | $\alpha$      | synaptic constant                                                      |


In [ ]:
# define constants 

# Shared 
C = 1.0 

# Neuron 1 
g_Na_1, g_K_1, g_L_1 = 120.0, 20.0, 0.3 
E_Na_1, E_K_1, E_L_1 = 50.0, -77.0, -54.4 
V0_m_1, V0_h_1, V0_n_1 = -40.0, -60.0, -55.0
dV0_m_1, dV0_h_1, dV0_n_1 = 15.0, -15.0, 30.0
tau_m_1_0, tau_m_1_1 = 0.1, 0.4
tau_h_1_0, tau_h_1_1 = 1.0, 7.0
tau_n_1_0, tau_n_1_1 = 1.0, 5.0

# Neuron 2
g_Na_2, g_K_2, g_L_2 = 120.0, 20.0, 0.3 
E_Na_2, E_K_2, E_L_2 = 50.0, -77.0, -54.4 
V0_m_2, V0_h_2, V0_n_2 = -40.0, -60.0, -55.0
dV0_m_2, dV0_h_2, dV0_n_2 = 15.0, -15.0, 30.0
tau_m_2_0, tau_m_2_1 = 0.1, 0.4
tau_h_2_0, tau_h_2_1 = 1.0, 7.0
tau_n_2_0, tau_n_2_1 = 1.0, 5.0

# Neuron 3
g_Na_3, g_K_3, g_L_3 = 120.0, 20.0, 0.3 
E_Na_3, E_K_3, E_L_3 = 50.0, -77.0, -54.4 
V0_m_3, V0_h_3, V0_n_3 = -40.0, -60.0, -55.0
dV0_m_3, dV0_h_3, dV0_n_3 = 15.0, -15.0, 30.0
tau_m_3_0, tau_m_3_1 = 0.1, 0.4
tau_h_3_0, tau_h_3_1 = 1.0, 7.0
tau_n_3_0, tau_n_3_1 = 1.0, 5.0

# Synapses
g_12 = 0.35
g_23 = 0.27
g_32 = 0.215
g_31 = 0.203

# what are E_rev's 
E_rev_e = 0.0 
E_rev_i = -80.0

tau1 = 1.0
tau2 = 3.0
S1 = 3.0/2.0
S2 = 5.0/3.0

In [ ]:
# functions 

def S0(V, V0=-0.5, dV0=5.0): 
    """Update synapse baseline S(V)"""
    return 0.5*(1.0 + math.tanh((V-V0)/dV0))

def tau(V, t0, t1, V0, dV0):
    """Denominator of update function for m(t), h(t), n(t)""" 
    return t0 + t1*(1.0 - math.tanh((V-V0)/dV0)**2)

def g_syn(V,V0,dV0): 
    """Numerator of update function for m(t), h(t), n(t)"""
    return 0.5 + 0.5 * math.tanh((V-V0)/dV0)

def dmdt(V):
    pass

def dhdt(V):
    pass

def dndt(V):
    pass

def dSdt(V): 
    pass

def dVdt(V, m, h, n, S): 
    pass

def dfdt(states): 
    dV1dt = dVdt(states) 
    dS12 = dtdSdt(states[0])
    pass 
    dstatesdt_lst = [dV1dt, dS12 ]
    return dstatesdt_lst

### TODO: Second, we need to create the simple HH neuron


In [ ]:
class HHNeuron(): 
    def __init__(self, g_Na_0, g_K_0, g_L_0, S_0_alpha, S_t_tau, V_baseline): 
        # gating constants
        self.g_Na = g_Na_0
        self.g_K = g_K_0
        self.g_L = g_L_0
        # S constants 
        self.alpha = S_0_alpha
        self.tau = S_t_tau
        self.V_0 = V_baseline
    
    # utility function 
    def S_0(self, V):
        pass

    def S_Vt(self, V_t, S_1): 
        pass

    # TODO: what are the S functions, and how do they interact

    def activate(self, I_pass, I_stim, 
                    V_t, m_t, h_t, n_t):
        """
        Change neuron's Na concentration and stuff
        
        Parameters: 
            I_pass: passing current from other neurons 
            I_stim: current from outside stimulant
            V_t: membrane voltage of the neuron
            m_t, h_t, n_t: gating variables 

        Return: 
            C $\frac{dV(t)}{dt}: activation of the neuron
        """
        pass

